In [1]:
import pandas as pd
import matplotlib.pyplot

# Trabalho Final de Aprendizado de Máquina - Learning of Legends
---
_O quanto um modelo de aprendizado supervisionado é capaz de prever o time vencedor de uma partida profissional de League of Legends, baseando-se apenas em informações do começo do jogo? Isto é, uma vantagem criada no começo do jogo e composições de equipe são o suficiente para ditar o resultado de uma partida? Ainda, há ligas profissionais cujos resultados podemos prever mais facilmente? Modelos treinados com partidas de todas as ligas são geralmente bons ou é melhor treinar modelos específicos para cada liga?_

---

Como conjunto de dados, temos informações de partidas de ligas profissionais e campeonatos internacionais oficiais disputadas entre 2015 e 2017, totalizando 7620 observações. (Disponível em https://www.kaggle.com/chuckephron/leagueoflegends - Acesso em 25/06/2019) As informações incluem os campeões escolhidos por ambos os times, e valores como diferença de ouro e conquista de objetivos por minuto de jogo. 

Usando todos os atributos disponíveis sobre as partidas, a previsão do time vencedor torna-se trivial. O objetivo do trabalho é investigar a capacidade de solução desse problema a partir da abordagem de aprendizado supervisionado, fazendo uso de um conjunto limitado de informação.

---

## Tratamento dos Dados



In [2]:
lol = pd.read_csv("data/LeagueOfLegends.csv")
lol.head()

,League,Year,Season,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff,...,redMiddleChamp,goldredMiddle,redADC,redADCChamp,goldredADC,redSupport,redSupportChamp,goldredSupport,redBans,Address
0,NALCS,2015,Spring,Season,TSM,1,0,C9,40,"[0, 0, -14, -65, -268, -431, -488, -789, -494,...",...,Fizz,"[475, 475, 552, 842, 1178, 1378, 1635, 1949, 2...",Sneaky,Sivir,"[475, 475, 532, 762, 1097, 1469, 1726, 2112, 2...",LemonNation,Thresh,"[515, 515, 577, 722, 911, 1042, 1194, 1370, 14...","['Tristana', 'Leblanc', 'Nidalee']",http://matchhistory.na.leagueoflegends.com/en/...
1,NALCS,2015,Spring,Season,CST,0,1,DIG,38,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,...",...,Azir,"[475, 475, 552, 786, 1097, 1389, 1660, 1955, 2...",CoreJJ,Corki,"[475, 475, 532, 868, 1220, 1445, 1732, 1979, 2...",KiWiKiD,Annie,"[515, 515, 583, 752, 900, 1066, 1236, 1417, 15...","['RekSai', 'Janna', 'Leblanc']",http://matchhistory.na.leagueoflegends.com/en/...
2,NALCS,2015,Spring,Season,WFX,1,0,GV,40,"[0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ...",...,Azir,"[475, 475, 533, 801, 1006, 1233, 1385, 1720, 1...",Cop,Corki,"[475, 475, 533, 781, 1085, 1398, 1782, 1957, 2...",BunnyFuFuu,Janna,"[515, 515, 584, 721, 858, 1002, 1168, 1303, 14...","['Leblanc', 'Zed', 'RekSai']",http://matchhistory.na.leagueoflegends.com/en/...
3,NALCS,2015,Spring,Season,TIP,0,1,TL,41,"[0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ...",...,Lulu,"[475, 475, 532, 771, 1046, 1288, 1534, 1776, 2...",KEITH,KogMaw,"[475, 475, 532, 766, 1161, 1438, 1776, 1936, 2...",Xpecial,Janna,"[515, 515, 583, 721, 870, 1059, 1205, 1342, 15...","['RekSai', 'Rumble', 'LeeSin']",http://matchhistory.na.leagueoflegends.com/en/...
4,NALCS,2015,Spring,Season,CLG,1,0,T8,35,"[40, 40, 44, -36, 113, 158, -121, -191, 23, 20...",...,Lulu,"[475, 475, 532, 807, 1042, 1338, 1646, 1951, 2...",Maplestreet8,Corki,"[475, 475, 532, 792, 1187, 1488, 1832, 2136, 2...",Dodo8,Annie,"[475, 475, 538, 671, 817, 948, 1104, 1240, 136...","['Rumble', 'Sivir', 'Rengar']",http://matchhistory.na.leagueoflegends.com/en/...


A primeira coisa que faremos é converter os valores de ouro por minuto. Como queremos trabalhar com informações de começo do jogo, vamos criar outro dataset apenas com informações como a diferença de ouro (computada por `Ouro do Time Azul - Ouro do Time Vermelho`), as eliminações e abate de torres e objetivos apenas aos 15 minutos de jogo.

In [11]:
lol.Year.apply(lambda y: str(y)) + " " + lol.Season

0       2015 Spring
1       2015 Spring
2       2015 Spring
3       2015 Spring
4       2015 Spring
5       2015 Spring
6       2015 Spring
7       2015 Spring
8       2015 Spring
9       2015 Spring
10      2015 Spring
11      2015 Spring
12      2015 Spring
13      2015 Spring
14      2015 Spring
15      2015 Spring
16      2015 Spring
17      2015 Spring
18      2015 Spring
19      2015 Spring
20      2015 Spring
21      2015 Spring
22      2015 Spring
23      2015 Spring
24      2015 Spring
25      2015 Spring
26      2015 Spring
27      2015 Spring
28      2015 Spring
29      2015 Spring
           ...     
7590    2018 Spring
7591    2018 Spring
7592    2018 Spring
7593    2018 Spring
7594    2018 Spring
7595    2018 Spring
7596    2018 Spring
7597    2018 Spring
7598    2018 Spring
7599    2018 Spring
7600    2018 Spring
7601    2018 Spring
7602    2018 Spring
7603    2018 Spring
7604    2018 Spring
7605    2018 Spring
7606    2018 Spring
7607    2018 Spring
7608    2018 Spring


In [13]:
try:
    lol_early = pd.read_csv("data/lol_early.csv")
except FileNotFoundError:
    lol_early = pd.DataFrame()
    # Informações genéricas
    lol_early["blueWins"] = lol.bResult
    lol_early["league"] = lol.League
    lol_early["season"] = lol.Year.apply(lambda y: str(y)) + " " + lol.Season
    lol_early["blueTeam"] = lol.blueTeamTag
    lol_early["redTeam"] = lol.redTeamTag
    lol_early["blueTop"] = lol.blueTopChamp
    lol_early["redTop"] = lol.redTopChamp
    lol_early["blueJungle"] = lol.blueJungleChamp
    lol_early["redJungle"] = lol.redJungleChamp
    lol_early["blueMiddle"] = lol.blueMiddleChamp
    lol_early["redMiddle"] = lol.redMiddleChamp
    lol_early["blueADC"] = lol.blueADCChamp
    lol_early["redADC"] = lol.redADCChamp
    lol_early["blueSupport"] = lol.blueSupportChamp
    lol_early["redSupport"] = lol.redSupportChamp
    
    # Extração da diferença de ouro/minuto em diferença de ouro aos 15 minutos, para cada posição
    lol_early["golddiffTeam15"] = lol.golddiff.apply(lambda s: int(s[1:-1].split(',')[14]))
    lol_early["golddiffTop15"] = lol.goldblueTop.apply(lambda s: int(s[1:-1].split(',')[14])) - lol.goldredTop.apply(lambda s: int(s[1:-1].split(',')[14]))
    lol_early["golddiffJungle15"] = lol.goldblueJungle.apply(lambda s: int(s[1:-1].split(',')[14])) - lol.goldredJungle.apply(lambda s: int(s[1:-1].split(',')[14]))
    lol_early["golddiffMiddle15"] = lol.goldblueMiddle.apply(lambda s: int(s[1:-1].split(',')[14])) - lol.goldredMiddle.apply(lambda s: int(s[1:-1].split(',')[14]))
    lol_early["golddiffADC15"] = lol.goldblueADC.apply(lambda s: int(s[1:-1].split(',')[14])) - lol.goldredADC.apply(lambda s: int(s[1:-1].split(',')[14]))
    lol_early["golddiffSupport15"] = lol.goldblueSupport.apply(lambda s: int(s[1:-1].split(',')[14])) - lol.goldredSupport.apply(lambda s: int(s[1:-1].split(',')[14]))
    
    # Extração de placar de eliminações aos 15 minutos
    lol_early["rKills15"] = lol.rKills.apply(lambda s: eval(s)).apply(lambda l: 0 if len(l)==0 else sum(1 for k in l if k[0] <= 15))
    lol_early["bKills15"] = lol.bKills.apply(lambda s: eval(s)).apply(lambda l: 0 if len(l)==0 else sum(1 for k in l if k[0] <= 15))
    
    # Extração do placar de Torres aos 15 minutos
    lol_early["rTowers15"] = lol.rTowers.apply(lambda s: eval(s)).apply(lambda l: 0 if len(l)==0 else sum(1 for t in l if t[0] <= 15))
    lol_early["bTowers15"] = lol.bTowers.apply(lambda s: eval(s)).apply(lambda l: 0 if len(l)==0 else sum(1 for t in l if t[0] <= 15))
    
    # Extração do placar de Inibidores aos 15 minutos
    lol_early["rInhibs15"] = lol.rInhibs.apply(lambda s: eval(s)).apply(lambda l: 0 if len(l)==0 else sum(1 for i in l if i[0] <= 15))
    lol_early["bInhibs15"] = lol.bInhibs.apply(lambda s: eval(s)).apply(lambda l: 0 if len(l)==0 else sum(1 for i in l if i[0] <= 15)) 
    
    lol_early.to_csv("data/lol_early.csv")

lol_early.head()

,blueWins,league,season,blueTeam,redTeam,blueTop,redTop,blueJungle,redJungle,blueMiddle,...,golddiffJungle15,golddiffMiddle15,golddiffADC15,golddiffSupport15,rKills15,bKills15,rTowers15,bTowers15,rInhibs15,bInhibs15
0,1,NALCS,2015 Spring,TSM,C9,Irelia,Gnar,RekSai,Elise,Ahri,...,-467,565,-98,-43,0,1,0,0,0,0
1,0,NALCS,2015 Spring,CST,DIG,Gnar,Irelia,Rengar,JarvanIV,Ahri,...,514,131,-192,117,3,5,0,0,0,0
2,1,NALCS,2015 Spring,WFX,GV,Renekton,Sion,Rengar,LeeSin,Fizz,...,55,1283,317,612,3,8,0,0,0,0
3,0,NALCS,2015 Spring,TIP,TL,Irelia,Gnar,JarvanIV,Nunu,Leblanc,...,-1019,-42,312,-97,4,2,0,0,0,0
4,1,NALCS,2015 Spring,CLG,T8,Gnar,Sion,JarvanIV,RekSai,Lissandra,...,-657,-164,474,37,1,1,2,1,0,0
